[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_toolbox/pdftools_toolbox_add_image_mask.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDKXT/latest/pdftools_toolbox-latest.tar.gz
%pip install ipython

# Add image mask to PDF
Place a rectangular image mask at a specified location of
a page. The image mask is a stencil mask to fill or mask
out the image per pixel.

In [ ]:
import io
import io
from pdftools_toolbox.pdf import Document, FileReference, Metadata, Page, PageCopyOptions, PageList
from pdftools_toolbox.geometry.real import Rectangle
from pdftools_toolbox.pdf.content import ColorSpace, ContentGenerator, IccBasedColorSpace, ImageMask, Paint, ProcessColorSpaceType
from pdftools_toolbox.pdf.navigation import ViewerSettings

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/BlankNoneNoTP.pdf'
input_file_path = 'BlankNoneNoTP.pdf'
prepare_file(input_url, input_file_path)
image_mask_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/TestFileResources/bird_bw.tif'
image_mask_path = 'bird_bw.tif'
prepare_file(image_mask_url, image_mask_path)
output_file_path = 'BlankWithImageMask.pdf'

In [ ]:
def copy_document_data(in_doc: Document, out_doc: Document):
    # Copy document-wide data

    # Output intent
    if in_doc.output_intent is not None:
        in_doc.output_intent = IccBasedColorSpace.copy(out_doc, in_doc.output_intent)

    # Metadata
    out_doc.metadata = Metadata.copy(out_doc, in_doc.metadata)

    # Viewer settings
    out_doc.viewer_settings = ViewerSettings.copy(out_doc, in_doc.viewer_settings)

    # Associated files (for PDF/A-3 and PDF 2.0 only)
    outAssociatedFiles = out_doc.associated_files
    for in_file_ref in in_doc.associated_files:
        outAssociatedFiles.append(FileReference.copy(out_doc, in_file_ref))

    # Plain embedded files
    out_embedded_files = out_doc.plain_embedded_files
    for in_file_ref in in_doc.plain_embedded_files:
        out_embedded_files.append(FileReference.copy(out_doc, in_file_ref))

In [ ]:
def add_image_mask(document: Document, page: Page, image_path: str, x: float, y: float):
    # Create content generator
    with ContentGenerator(page.content, False) as generator:

        # Load image from input path
        with io.FileIO(image_path, 'rb') as in_image_stream:
            # Create image mask object
            image_mask = ImageMask.create(document, in_image_stream)
            resolution = 150

            # Calculate rectangle for image
            size = image_mask.size
            rect = Rectangle(
                left=x,
                bottom=y,
                right=x + size.width * 72 / resolution,
                top=y + size.height * 72 / resolution
            )

            # Paint image mask into the specified rectangle
            generator.paint_image_mask(image_mask, rect, paint)

In [ ]:
try:
    # Set and check license key. If the license key is not valid, an exception is thrown.
    from pdftools_toolbox.sdk import Sdk
    Sdk.initialize("INSERT-LICENSE-KEY", None)

    # Open input document
    with io.FileIO(input_file_path, 'rb') as in_stream:
        with Document.open(in_stream, None) as in_doc:
    
            # Create output document
            with io.FileIO(output_file_path, 'wb+') as output_stream:
                with Document.create(output_stream, in_doc.conformance, None) as out_doc:
    
                    # Copy document-wide data
                    copy_document_data(in_doc, out_doc)
    
                    # Get the device color space
                    color_space = ColorSpace.create_process_color_space(out_doc, ProcessColorSpaceType.RGB)
    
                    # Create paint object
                    paint = Paint.create(out_doc, color_space, [1.0, 0.0, 0.0], None)
    
                    # Define page copy options
                    copy_options = PageCopyOptions()
    
                    # Copy first page, add image mask, and append to output document
                    out_page = Page.copy(out_doc, in_doc.pages[0], copy_options)
                    add_image_mask(out_doc, out_page, image_mask_path, 250, 150)
                    out_doc.pages.append(out_page)
    
                    # Copy remaining pages and append to output document
                    in_page_range = in_doc.pages[1:]
                    copied_pages = PageList.copy(out_doc, in_page_range, copy_options)
                    out_doc.pages.extend(copied_pages)

    print("Execution successful.")
except Exception as e:
    print(f"An error occurred: {e}")